In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
books=pd.read_csv("C:\\Users\\D Abhishek\\Desktop\\books.csv")
ratings=pd.read_csv("C:\\Users\\D Abhishek\\Desktop\\ratings.csv")
users=pd.read_csv("C:\\Users\\D Abhishek\\Desktop\\users.csv")


C:\Users\D Abhishek\AppData\Local\Temp\ipykernel_13284\2565892782.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books=pd.read_csv("C:\\Users\\D Abhishek\\Desktop\\books.csv")


In [ ]:
books

In [ ]:
ratings

In [ ]:
users

In [ ]:
books['Image-URL-M'][1]

In [ ]:
books.head()

In [ ]:
ratings.head()

In [ ]:
users.head()

In [ ]:
print(books.shape)
print(ratings.shape)
print(users.shape)

In [ ]:
books.isnull().sum()

In [ ]:
ratings.isnull().sum()

In [ ]:
users.isnull().sum()

In [ ]:
books.duplicated().sum()

In [ ]:
ratings.duplicated().sum()

In [ ]:

users.duplicated().sum()

In [ ]:
ratings_with_names =ratings.merge(books,on='ISBN')

In [ ]:
ratings_with_names

In [ ]:
num_rating_df = ratings_with_names.groupby('Book-Title').count()['Book-Rating'].reset_index()
num_rating_df.rename(columns={'Book-Rating':'num_ratings'},inplace=True)
num_rating_df

In [ ]:
avg_rating_df = ratings_with_names.groupby('Book-Title').mean()['Book-Rating'].reset_index()
avg_rating_df.rename(columns={'Book-Rating':'avg_rating'},inplace=True)
avg_rating_df

In [ ]:
popular_df = num_rating_df.merge(avg_rating_df,on='Book-Title')
popular_df

In [ ]:
popular_df = popular_df[popular_df['num_ratings']>=250].sort_values('avg_rating',ascending=False).head(50)


In [ ]:
popular_df = popular_df.merge(books,on='Book-Title').drop_duplicates('Book-Title')[['Book-Title','Book-Author','Image-URL-M','num_ratings','avg_rating']]


In [ ]:
popular_df['Image-URL-M'][0]

In [ ]:
x = ratings_with_names.groupby('User-ID').count()['Book-Rating'] > 200
padhe_likhe_users = x[x].index
filtered_rating = ratings_with_names[ratings_with_names['User-ID'].isin(padhe_likhe_users)]
y = filtered_rating.groupby('Book-Title').count()['Book-Rating']>=50
famous_books = y[y].index
final_ratings = filtered_rating[filtered_rating['Book-Title'].isin(famous_books)]
pt = final_ratings.pivot_table(index='Book-Title',columns='User-ID',values='Book-Rating')
pt.fillna(0,inplace=True)

In [ ]:
pt

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity_scores = cosine_similarity(pt)
similarity_scores.shape

In [ ]:
def recommend(book_name):
    # index fetch
    index = np.where(pt.index==book_name)[0][0]
    similar_items = sorted(list(enumerate(similarity_scores[index])),key=lambda x:x[1],reverse=True)[1:5]
    
    data = []
    for i in similar_items:
        item = []
        temp_df = books[books['Book-Title'] == pt.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Author'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Image-URL-M'].values))
        
        data.append(item)
    
    return data

In [ ]:
np.where(pt.index=="A Bend in the Road")[0][0]

In [ ]:
recommend('A Bend in the Road')

In [ ]:
pt.index[545]

In [ ]:
import pickle

In [ ]:
pickle.dump(popular_df,open('popular.pkl','wb'))
books.drop_duplicates('Book-Title')